In [33]:
# ===============
# Pipeline Setup (You can comment out the installation line below, after initial execution)
# ===============
# %pip install torch torchvision torchaudio transformers sentencepiece accelerate bitsandbytes

In [34]:
import os
import re
import json
import torch
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [35]:
# ------------------
# Helper Utilities
# ------------------

def safe_name(s: str) -> str:
    s = re.sub(r"[^A-Za-z0-9._\- ]+", "_", s)
    s = re.sub(r"\s+", "_", s)
    return s.strip("_")[:80] or "dependency"

def write_file(path: Path, content: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(content, encoding="utf-8")

In [36]:
# --------------
# Model Loader
# --------------

def load_model(model_name: str):
    """Load CodeLlama instruct model for local inference."""
    print(f"[info] Loading model: {model_name}")
    device = "cuda" if torch.cuda.is_available() else "cpu"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None,
        quantization_config=bnb_config
    )
    print(f"[info] Model loaded successfully on {device}")
    return model, tokenizer

In [39]:
llm_instruction = """
Your task is to generate a JUnit test that confirms reachability of the specified third party method within the target project.

You are given dependency information that describes:
1. The entry point method inside the project that eventually triggers the third party call.
2. The full method body of the entry point.
3. The fully qualified third party method that must be reached during execution.
4. The package that contains the third party method.
5. The path of intermediate calls that lead from the entry point to the third party method.
6. Either the full methods or sliced methods along that path depending on the technique being used.

Your goal is to produce a Java JUnit test that:
* Calls the entry point in a way that ensures execution flows through the provided path.
* Ensures the third party method is actually reached.
* Provides any required setup, mocks, stubs, or input values so the path executes without failure.
* Avoids unrelated logic and focuses only on reaching the third party call.

Output Requirements:
* Return only the Java code in a fenced code block labeled java i.e ```java```. Do not include explanations or commentary!!!
* The test must compile.
* Use realistic values for parameters so the call chain executes.
* If mocking is needed to force reachability, add appropriate mocks or placeholders.
"""

In [41]:
def test_generator(
    prompt: str,
    dependency_data: dict,
    model_name: str,
    output_root: str,
    technique: str = "",
):
    # technique: entry_point, full_methods, sliced
    timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")

    qualified_name = dependency_data.get("thirdPartyMethod") or dependency_data.get("thirdPartyPackage") or "unknown"
    entry_point = dependency_data.get("entryPoint", "entry")
    # make filename unique: include entry point and timestamp
    safe_filename = safe_name(f"{entry_point}__{qualified_name}") + f"_{timestamp}.java"

    folder = Path(output_root)
    folder.mkdir(parents=True, exist_ok=True)

    messages = [
        {"role": "system", "content": llm_instruction},
        {"role": "user", "content": prompt},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=False,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    print("[info] Generating message...")
    outputs = model.generate(**inputs, max_new_tokens=2048)
    print("[info] Generation complete.")

    # robustly extract generated token ids
    # supports HF generate returning tensor or ModelOutput with .sequences
    if isinstance(outputs, torch.Tensor):
        seq = outputs[0]
    else:
        seq = getattr(outputs, "sequences", None)
        seq = seq[0] if seq is not None else outputs[0]

    start_idx = inputs["input_ids"].shape[-1]
    generated_ids = seq[start_idx:].cpu().numpy().tolist()
    decoded = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    # do NOT remove prompt from decoded. use decoded as-is
    result_text = decoded or ""
    print("\n generated response\n")
    print(result_text[:1000])  # print a short preview
    print("\nresponse end\n")

    # prefer code fence extraction if present
    m = re.search(r"```(?:java)?\s*(.*?)\s*```", result_text, re.DOTALL)
    java_code = (m.group(1).strip() if m else result_text).strip()

    if not java_code:
        # fallback safe placeholder so files are still created and diagnosable
        java_code = "// [warning] model returned empty output\n" + (result_text or "// no content")

    file_path = folder / safe_filename
    file_path.write_text(java_code, encoding="utf-8")

    print(f"[done] Output written to {file_path.resolve()}")

    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    return {
        "folder": str(folder.resolve()),
        "java_file": str(file_path.resolve()),
    }


# Automating test generation for etheo code examples 

 - ensure to upload the example JSON files from etheo github repo

In [45]:
def generate_etheo_third_party_tests(
    project_name: str,
    model_name: str,
    examples_root: str,
    output_root: str,
):
    base = Path(examples_root) / project_name

    technique_files = {
        "entry_point": ("third_party_apis_entry_point.json", "entryPointPaths"),
        "full_methods": ("third_party_apis_full_methods.json", "fullMethodsPaths"),
        "sliced": ("third_party_apis_sliced.json", "slicedPaths"),
    }

    results = []

    for technique, (filename, json_key) in technique_files.items():
        file_path = base / filename
        if not file_path.exists():
            print(f"[skip] missing file {filename}")
            continue

        try:
            raw = file_path.read_text(encoding="utf8")
            parsed = json.loads(raw)
            deps = parsed.get(json_key, [])
        except Exception as e:
            print(f"[error] failed to load {filename}: {e}")
            continue

        if not isinstance(deps, list) or not deps:
            print(f"[skip] no data in {filename}")
            continue

        technique_output_dir = (
            Path(output_root) / project_name / technique
        )
        technique_output_dir.mkdir(parents=True, exist_ok=True)

        for idx, dep in enumerate(deps):
            try:
                dependency_data = dep
                dep_json = json.dumps(dependency_data, indent=2, ensure_ascii=False)

                prompt = (
                    f"Dependency Data\n"
                    f"{dep_json}\n\n"
                )

                out = test_generator(
                    prompt=prompt,
                    dependency_data=dependency_data,
                    model_name=model_name,
                    output_root=str(technique_output_dir),
                    technique=technique,
                )

                results.append({
                    "technique": technique,
                    "file": filename,
                    "index": idx,
                    "entryPoint": dependency_data.get("entryPoint"),
                    "thirdPartyMethod": dependency_data.get("thirdPartyMethod"),
                    "result": out,
                })

                print(f"[ok] {filename} #{idx} for technique {technique}")

            except Exception as e:
                print(f"[error] {filename} #{idx} failed: {e}")
                continue

    return results

In [46]:
results = generate_etheo_third_party_tests(
    project_name="pdfbox-data",
    model_name="codellama/CodeLlama-7b-Instruct-hf",
    examples_root="/kaggle/input/etheo-project/examples",
    output_root="/kaggle/working/generated_tests",
)


print("\n")
print(results)
print("\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[info] Generating message...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[info] Generation complete.

 generated response

Here is a JUnit test that confirms reachability of the specified third party method within the target project:
```java
import org.junit.Test;
import org.apache.commons.logging.LogFactory;
import static org.junit.Assert.assertNotNull;

public class ReachabilityTest {

    @Test
    public void testReachability() {
        // Call the entry point method to ensure execution flows through the provided path
        PDPolylineAppearanceHandler.generateNormalAppearance();

        // Ensure the third party method is actually reached
        Log log = LogFactory.getLog("org.apache.pdfbox.pdmodel.interactive.annotation.handlers.PDPolylineAppearanceHandler");
        assertNotNull(log);
    }
}
```
This test calls the entry point method `PDPolylineAppearanceHandler.generateNormalAppearance()` to ensure execution flows through the provided path. It then uses the `LogFactory.getLog()` method to retrieve the log object for the `PDPolylineAppearanceH

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[info] Generation complete.

 generated response

Here is a JUnit test that confirms reachability of the third party method `org.apache.commons.logging.LogFactory.getLog` within the target project:
```java
import org.junit.Test;
import org.apache.pdfbox.rendering.GlyphCache;
import org.apache.commons.logging.LogFactory;

public class GlyphCacheTest {
    @Test
    public void testGetPathForCharacterCode() {
        // Set up mocks or placeholders as needed
        // ...

        // Call the entry point method to trigger execution
        GeneralPath path = GlyphCache.getPathForCharacterCode(123);

        // Assert that the third party method was reached
        LogFactory logFactory = LogFactory.getLog(GlyphCache.class);
        assertNotNull(logFactory);
    }
}
```
This test calls the `GlyphCache.getPathForCharacterCode` method, which eventually triggers the execution of the third party method `org.apache.commons.logging.LogFactory.getLog`. The test then asserts that the third part

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[info] Generation complete.

 generated response

```java
import org.apache.pdfbox.contentstream.PDFStreamEngine;
import org.apache.pdfbox.contentstream.operator.text.MoveText;
import org.apache.commons.logging.LogFactory;
import org.junit.Test;

public class ReachabilityTest {
    @Test
    public void testReachability() {
        // Setup
        PDFStreamEngine engine = new PDFStreamEngine();
        engine.processOperator("SomeOperation", null);

        // Act
        engine.processOperator("MoveText", null);

        // Assert
        LogFactory.getLog("org.apache.pdfbox.contentstream.operator.text.MoveText");
    }
}
```
This test calls the `processOperator` method on the `PDFStreamEngine` class with the `MoveText` operator, which eventually leads to the `org.apache.commons.logging.LogFactory.getLog` method being called. The test asserts that the `getLog` method is called by checking the log output.

Note that this test is a simplified example and may not be representative of th

KeyboardInterrupt: 